In [1]:
import polars as pl
from pathlib import Path

In [2]:
dpath = Path('../../dataset')

dtype = 'small'
articles = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/articles.parquet')

behaviors_train = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/train/behaviors.parquet')
history_train = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/train/history.parquet')

behaviors_val = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/validation/behaviors.parquet')
history_val = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/validation/history.parquet')

In [3]:
from polimi.utils._custom import load_best_optuna_params
study_name='RP3betaRecommender-ner-small-ndcg10'
params = load_best_optuna_params(study_name)
params

{'topK': 93,
 'normalize_similarity': True,
 'alpha': 0.3803092963950007,
 'beta': 0.2431345797113146}

In [4]:
from polimi.utils._custom import load_sparse_csr

ner_path = dpath.joinpath('urm').joinpath('ner').joinpath('small')
URM_train = load_sparse_csr(ner_path.joinpath('URM_train.npz'))
URM_val =  load_sparse_csr(ner_path.joinpath('URM_validation.npz'))

File loaded at: ../../dataset/urm/ner/small/URM_train.npz
File loaded at: ../../dataset/urm/ner/small/URM_validation.npz


In [5]:
Path('.').resolve()

PosixPath('/Users/lorecampa/Desktop/Projects/RecSysChallenge2024/notebooks/urm')

In [6]:
from polimi.utils._urm import train_recommender
from RecSys_Course_AT_PoliMi.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

rec = train_recommender(URM_train, RP3betaRecommender, params, file_name='test', output_dir=Path('.'))

RP3betaRecommender: URM Detected 3685 (19.6%) users with no interactions.
RP3betaRecommender: URM Detected 18909 (43.2%) items with no interactions.
RP3betaRecommender: Similarity column 43774 (100.0%), 673.25 column/sec. Elapsed time 1.08 min


TypeError: unsupported operand type(s) for +: 'PosixPath' and 'str'

In [4]:
from RecSys_Course_AT_PoliMi.Evaluation.Evaluator import EvaluatorHoldout

cutoff = 10
metric = 'NDCG'
evaluator = EvaluatorHoldout(URM_val, cutoff_list=[cutoff], exclude_seen=False)

EvaluatorHoldout: Ignoring 3488 (18.5%) Users that have less than 1 test interactions


In [ ]:
from RecSys_Course_AT_PoliMi.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender

rec_instance = PureSVDRecommender(URM_train)
params = {'num_factors': 577}
rec_instance.fit(**params)
result_df, _ = evaluator.evaluateRecommender(rec_instance)
result_df.loc[cutoff][metric.upper()]

In [ ]:
from RecSys_Course_AT_PoliMi.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender

rec_instance = UserKNNCFRecommender(URM_train)
params = {'topK': 467, 'shrink': 95, 'similarity': 'dice', 'normalize': True}
rec_instance.fit(**params)
result_df, _ = evaluator.evaluateRecommender(rec_instance)
result_df.loc[cutoff][metric.upper()]

In [ ]:
from polimi.utils._urm import _build_batch_ner_interactions, build_articles_with_processed_ner, build_user_id_mapping, build_ner_mapping
ap = build_articles_with_processed_ner(articles)
user_id_mapping = build_user_id_mapping(history_train.vstack(history_val))
ner_mapping = build_ner_mapping(ap)
ner_interactions = _build_batch_ner_interactions(history_train, ap, user_id_mapping, ner_mapping, 'article_id_fixed')